In [1]:
import numpy as np
import pandas as pd

In [2]:
coin_ids = {
    'Bitcoin': 1442,
    'Bitcoin Cash': 1445,
    'Bitcoin Gold': 1456,
    'Cardano': 1446,
    'Dash': 1453,
    'Dogecoin': 1447,
    'Eos': 1452,
    'Ethereum': 1443,
    'Ethereum Classic': 1457,
    'Iota': 1451,
    'Lisk': 1460,
    'Litecoin': 1448,
    'Monero': 1454,
    'NEMcoin': 1447,
    'Neo': 1449,
    'Ripple': 1444,
    'Stellar': 1450,
    'Tether': 1474,
    'Tron': 1455,
    'Zcash': 1465,
}

id_coins = {v: k for k, v in coin_ids.items()}

In [3]:
iterator_full_data = pd.read_csv('data/datathon.csv', iterator=True, chunksize=100000, parse_dates = ['time'])
subset_full_data = pd.concat([chunk[chunk.refID_coin.isin(coin_ids.values())] for chunk in iterator_full_data])

data from http://www.forextester.com/data/datasources

In [6]:
%ls data

coin__ids.csv  datathon.csv  EURUSD.csv  price_data.csv  XAUUSD.csv


In [75]:
eurusd = pd.read_csv('data/EURUSD.csv', parse_dates = ['<DTYYYYMMDD>'])
eurusd = eurusd[eurusd['<TIME>'] % 500 == 0][['<DTYYYYMMDD>', '<TIME>', '<CLOSE>']]
eurusd = eurusd.rename({'<DTYYYYMMDD>': 'date', '<TIME>': 'minute', '<CLOSE>': 'eurusd'}, axis=1)
eurusd.minute = eurusd.minute/100

In [76]:
xauusd = pd.read_csv('data/XAUUSD.csv', parse_dates = ['<DTYYYYMMDD>'])
xauusd = xauusd[xauusd['<TIME>'] % 500 == 0][['<DTYYYYMMDD>', '<TIME>', '<CLOSE>']]
xauusd = xauusd.rename({'<DTYYYYMMDD>': 'date', '<TIME>': 'minute', '<CLOSE>': 'xauusd'}, axis=1)
xauusd.minute = xauusd.minute/100

### write only filtered files to repo, so that people don't download huge files:

In [85]:
xauusd[xauusd.date > '2017-01-01'].to_csv('data/filtered_xauusd.csv')
eurusd[eurusd.date > '2017-01-01'].to_csv('data/filtered_eurusd.csv')

### helper columns for the merge:

In [77]:
subset_full_data['minute'] = (subset_full_data.time.dt.minute + subset_full_data.time.dt.hour*100).astype('int')
subset_full_data['date'] = subset_full_data.time.dt.date.astype('<M8[ns]')

### merge:

In [78]:
subset_full_data = pd.merge(subset_full_data, eurusd, how = 'left', on = ['date', 'minute'])
subset_full_data = pd.merge(subset_full_data, xauusd, how = 'left', on = ['date', 'minute'])

### fill na values (sat, sunday):

In [79]:
subset_full_data.fillna(method = 'ffill', inplace=True)